In [ ]:
class Problem:
    '''
    Abstract base class for problem formulation.
    It declares the expected methods to be used by a search algorithm.
    All the methods declared are just placeholders that throw errors if not overriden by child "concrete" classes!
    '''

    def __init__(self):
        '''Constructor that initializes the problem. Typically used to setup the initial state and, if applicable, the goal state.'''
        self.init_state = None

    def actions(self, state):
        '''Returns an iterable with the applicable actions to the given state.'''
        raise NotImplementedError

    def result(self, state, action):
        '''Returns the resulting state from applying the given action to the given state.'''
        raise NotImplementedError

    def goal_test(self, state):
        '''Returns whether or not the given state is a goal state.'''
        raise NotImplementedError

    def step_cost(self, state, action):
        '''Returns the step cost of applying the given action to the given state.'''
        raise NotImplementedError


In [ ]:
class Node:
    '''Node data structure for search space bookkeeping.'''

    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def __repr__(self):
        return f"Node(state={self.state}, path_cost={self.path_cost})"

    @staticmethod
    def root(state):
        return Node(state)

    @staticmethod
    def child(problem, parent, action):
        state = problem.result(parent.state, action)
        path_cost = parent.path_cost + problem.step_cost(parent.state, action)
        return Node(state, parent, action, path_cost)

def __lt__(self, other):
        return self.path_cost < other.path_cost

def solution(node):
    '''A method to extract the sequence of actions representing the solution from the goal node.'''
    actions = []
    cost = node.path_cost
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions, cost

In [ ]:
from shutil import get_terminal_size
terminal_width, _ = get_terminal_size()

_visualizers = {}

def _default_visualizer(_, state):
    '''Generic visualizer for unknown problems.'''
    print(state)

class Visualizer:
    '''Visualization and printing functionality encapsulation.'''

    def __init__(self, problem):
        '''Constructor with the problem to visualize.'''
        self.problem = problem
        self.counter = 0

    def visualize(self, frontier):
        '''Visualizes the frontier at every step.'''
        self.counter += 1
        print(f'Frontier at step {self.counter}')
        for node in frontier:
            print()
            _visualizers.get(type(self.problem), _default_visualizer)(self.problem, node.state)
        print('-' * terminal_width)



# **TO MODIFY ANYTHING OF THE MAZE **

In [ ]:
class TreasureHunt:
    def __init__(self, grid_size=(32, 32), treasure_location=(31, 31), initial_state=(0, 0),obstacles=None):
        self.grid_size = grid_size
        self.treasure_location = treasure_location
        self.init_state = initial_state
        self.obstacles = obstacles if obstacles is not None else []



    def actions(self, state):
        possible_actions = ['up', 'down', 'left', 'right']
        valid_actions = []

        for action in possible_actions:
            new_row, new_col = self.result(state, action)
            if (0 <= new_row < self.grid_size[0] and
                0 <= new_col < self.grid_size[1] and
                (new_row, new_col) not in self.obstacles):
                valid_actions.append(action)

        return valid_actions

    def result(self, state, action):
        row, col = state

        if action == 'up':
            row -= 1
        elif action == 'down':
            row += 1
        elif action == 'left':
            col -= 1
        elif action == 'right':
            col += 1

        return row, col

    def goal_test(self, state):
        return state == self.treasure_location

    def step_cost(self, state, action):
        return 1

def _treasure_hunt_visualizer(problem, state):
        """Custom visualizer for the Treasure Hunt problem."""
        grid = [['.' for _ in range(problem.grid_size[1])] for _ in range(problem.grid_size[0])]
        grid[problem.treasure_location[0]][problem.treasure_location[1]] = 'T'
        grid[state[0]][state[1]] = 'A'

        for obstacle in problem.obstacles:
            grid[obstacle[0]][obstacle[1]] = 'X'

        for row in grid:
            print(' '.join(row))

_visualizers[TreasureHunt] = _treasure_hunt_visualizer

# ***HILL CLIMBING METHOD***

In [ ]:
class TreasureHunt:
    def __init__(self, grid_size=(32, 32), treasure_location=(31, 31), initial_state=(0, 0),obstacles=None):
        self.grid_size = grid_size
        self.treasure_location = treasure_location
        self.init_state = initial_state
        self.obstacles = obstacles if obstacles is not None else []



    def actions(self, state):
        possible_actions = ['up', 'down', 'left', 'right']
        valid_actions = []

        for action in possible_actions:
            new_row, new_col = self.result(state, action)
            if (0 <= new_row < self.grid_size[0] and
                0 <= new_col < self.grid_size[1] and
                (new_row, new_col) not in self.obstacles):
                valid_actions.append(action)

        return valid_actions

    def result(self, state, action):
        row, col = state

        if action == 'up':
            row -= 1
        elif action == 'down':
            row += 1
        elif action == 'left':
            col -= 1
        elif action == 'right':
            col += 1

        return row, col

    def heuristic(self, state):
        return abs(state[0] - self.treasure_location[0]) + abs(state[1] - self.treasure_location[1])

    def goal_test(self, state):
        return state == self.treasure_location

    def step_cost(self, state, action):
        return 1  # Assume each move has a cost of 1

def _treasure_hunt_visualizer(problem, state):
        """Custom visualizer for the Treasure Hunt problem."""
        grid = [['.' for _ in range(problem.grid_size[1])] for _ in range(problem.grid_size[0])]
        grid[problem.treasure_location[0]][problem.treasure_location[1]] = 'T'
        grid[state[0]][state[1]] = 'A'

        for obstacle in problem.obstacles:
            grid[obstacle[0]][obstacle[1]] = 'X'

        for row in grid:
            print(' '.join(row))

_visualizers[TreasureHunt] = _treasure_hunt_visualizer


In [ ]:
def hill_climbing(problem):
    """Function to perform Hill Climbing search on a given problem."""
    current = Node.root(problem.init_state)

    _treasure_hunt_visualizer(problem, problem.init_state)
    print('-' * terminal_width)

    while True:
        neighbors = [Node.child(problem, current, action)
                    for action in problem.actions(current.state)]
        if not neighbors:
            break
        next_node = min(neighbors, key=lambda node: problem.heuristic(node.state))

        if problem.heuristic(next_node.state) >= problem.heuristic(current.state):
            break

        current = next_node

        _treasure_hunt_visualizer(problem, current.state)
        print('-' * terminal_width)

    return current

In [ ]:
def run_hill_climbing():
    print("Starting Treasure Hunt with Hill Climbing...")
    problem = TreasureHunt(grid_size=(20, 20), initial_state=(0, 0), treasure_location=(19, 19))

    goal_node = hill_climbing(problem)

    if problem.goal_test(goal_node.state):
        actions, cost = solution(goal_node)
        print("\nSuccess! Path found:", actions)
        print("Total cost:", cost)
    else:
        print("\nNo path found - got stuck in local minimum!")

run_hill_climbing()

Starting Treasure Hunt with Hill Climbing...
A . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . T
----------------------------------------------------------------------------------------------------
. . . . . . . . . . . . . . . . . . . .
A . . . . . . 

In [ ]:
import time

def hill_climbing_with_performance(problem):
    """Hill Climbing search with performance measurement."""
    start_time = time.time()

    current = Node.root(problem.init_state)
    nodes_explored = 1

    _treasure_hunt_visualizer(problem, problem.init_state)
    print('-' * terminal_width)

    while True:
        neighbors = [Node.child(problem, current, action)
                    for action in problem.actions(current.state)]
        nodes_explored += len(neighbors)

        if not neighbors:
            break
        next_node = min(neighbors, key=lambda node: problem.heuristic(node.state))

        if problem.heuristic(next_node.state) >= problem.heuristic(current.state):
            break

        current = next_node

        _treasure_hunt_visualizer(problem, current.state)
        print('-' * terminal_width)

        if problem.goal_test(current.state):

            end_time = time.time()
            execution_time = end_time - start_time
            actions, cost = solution(current)
            return actions, cost, nodes_explored, execution_time

    end_time = time.time()
    execution_time = end_time - start_time
    return None, None, nodes_explored, execution_time

def run_hill_climbing_with_performance():
    print("Starting Treasure Hunt with Hill Climbing...")
    problem = TreasureHunt(grid_size=(20, 20), initial_state=(0, 0), treasure_location=(19, 19))

    solution_path, cost, nodes_explored, execution_time = hill_climbing_with_performance(problem)

    if solution_path:
        print("\nSuccess! Path found:", solution_path)
        print("Total cost:", cost)
        print(f"Nodes explored: {nodes_explored}")
        print(f"Execution time: {execution_time:.6f} seconds")
    else:
        print("\nNo path found - got stuck in local minimum!")
        print(f"Nodes explored: {nodes_explored}")
        print(f"Execution time: {execution_time:.6f} seconds")

run_hill_climbing_with_performance()


Starting Treasure Hunt with Hill Climbing...
A . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . T
----------------------------------------------------------------------------------------------------
. . . . . . . . . . . . . . . . . . . .
A . . . . . . 

# ***A STAR***

In [ ]:
class Node:
    def __init__(self, state, parent, action, path_cost):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def __lt__(self, other):
        """Define how to compare nodes using their path cost."""
        return self.path_cost < other.path_cost

    @classmethod
    def root(cls, init_state):
        return cls(init_state, None, None, 0)

    @classmethod
    def child(cls, problem, parent, action):
        return cls(
            problem.result(parent.state, action),
            parent,
            action,
            parent.path_cost + problem.step_cost(parent.state, action)
        )

def solution(node):
    actions = []
    cost = node.path_cost
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions, cost

In [ ]:
def memoize(fn, attr_name):
    """Memoizes function results by storing them in an attribute of the problem object."""
    cache = {}

    def memoized_fn(node, problem):
        if node.state not in cache:
            cache[node.state] = fn(node, problem)
        return cache[node.state]

    return memoized_fn

In [ ]:
from heapq import heappush, heappop
# A* search function
def astar_search(problem, h=None, verbose=False):
    """A* search with optional verbose output."""
    h = memoize(h or problem.h, 'h_cache')

    node = Node.root(problem.init_state)
    frontier = [(h(node, problem), 0, node)]
    explored = set()
    max_frontier_size = 1

    visualizer = Visualizer(problem) if verbose else None

    if verbose:
        visualizer.visualize([node])

    while frontier:
        _, _, node = heappop(frontier)
        if problem.goal_test(node.state):
            return solution(node), max_frontier_size

        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            f = child.path_cost + h(child, problem)
            if child.state not in explored and all(c.state != child.state for _, _, c in frontier):
                heappush(frontier, (f, len(explored), child))

        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize([n for _, _, n in frontier])

    return None, max_frontier_size

In [ ]:
def heuristic(node, problem):
    """Heuristic function for Treasure Hunt, adapted to use the node correctly."""
    agent_x, agent_y = node.state

    treasure_x, treasure_y = problem.treasure_location
    return abs(agent_x - treasure_x) + abs(agent_y - treasure_y)

In [ ]:
obstacles = [(1, 1), (2, 2), (3, 3)]
treasure_hunt_problem = TreasureHunt(grid_size=(20, 20), initial_state=(0, 0), treasure_location=(19, 19), obstacles=obstacles)
solution_steps, max_frontier_size = astar_search(treasure_hunt_problem, heuristic, verbose=True)
print(f"Solution steps: {solution_steps}")
print(f"Max frontier size: {max_frontier_size}")

Streaming output truncated to the last 5000 lines.
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . A . . . . . . T

. . . . . . . . . . . . . . . . . . . .
. X . . . . . . . . . . . . . . . . . .
. . X . . . . . . . . . . . . . . . . .
. . . X . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . A . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . 

# ***DFS***

In [ ]:
import time
from collections import deque

def dfs_search(problem, verbose=False):
    """Depth-First Search algorithm."""
    node = Node.root(problem.init_state)
    frontier = deque([node])
    explored = set()
    max_frontier_size = 1

    visualizer = Visualizer(problem) if verbose else None

    if verbose:
        visualizer.visualize([node])

    start_time = time.time()  # Start the timer

    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            end_time = time.time()  # End the timer
            elapsed_time = end_time - start_time
            return solution(node), max_frontier_size, elapsed_time

        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored and all(c.state != child.state for c in frontier):
                frontier.append(child)
                if problem.goal_test(child.state):
                    end_time = time.time()  # End the timer
                    elapsed_time = end_time - start_time
                    return solution(child), max_frontier_size, elapsed_time

        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize([n for n in frontier])

    return None, max_frontier_size, 0  # If no solution, return 0 for elapsed time

obstacles = [(1, 1), (2, 2), (3, 3)]
problem = TreasureHunt(grid_size=(20, 20), initial_state=(0, 0), treasure_location=(19, 19),obstacles=obstacles)
solution_path, max_frontier_size, elapsed_time = dfs_search(problem, verbose=True)

if solution_path:
    actions, cost = solution_path
    print("Solution found with actions:", actions)
    print("Total cost:", cost)
    print("Maximum size of the frontier:", max_frontier_size)
    print("Elapsed time:", elapsed_time)
else:
    print("No solution found.")

Streaming output truncated to the last 5000 lines.
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . A . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . T

. . . . . . . . . . . . . . . . . . . .
. X . . . . . . . . . . . . . . . . . .
. . X . . . . . . . . . . . . . . . . .
. . . X . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . 

# ***BFS***

In [ ]:
import time
from collections import deque

def bfs_search(problem, verbose=False):
    """Breadth-First Search algorithm."""
    node = Node.root(problem.init_state)
    frontier = deque([node])
    explored = set()
    visualizer = Visualizer(problem) if verbose else None

    start_time = time.time()

    while frontier:
        if verbose:
            visualizer.visualize(frontier)

        node = frontier.popleft()
        if problem.goal_test(node.state):
            end_time = time.time()
            execution_time = end_time - start_time
            print(f"Algorithm executed in {execution_time:.4f} seconds.")
            return solution(node), node.path_cost

        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored and all(c.state != child.state for c in frontier):
                frontier.append(child)

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Algorithm execute in {execution_time:.4f} seconds.")

    return None, 0

obstacles = [(1, 1), (4, 2), (3, 3)]
problem = TreasureHunt(grid_size=(20, 20), initial_state=(0, 0), treasure_location=(19, 19),obstacles=obstacles)
solution_path, max_frontier_size = bfs_search(problem, verbose=True)

if solution_path:
    actions, cost = solution_path
    print("Solution found with actions:", actions)
    print("Total cost:", cost)
else:
    print("No solution found.")


Streaming output truncated to the last 5000 lines.
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . A . . . . . .
. . . . . . . . . . . . . . . . . . . T

. . . . . . . . . . . . . . . . . . . .
. X . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . X . . . . . . . . . . . . . . . .
. . X . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . A . . . . .
. . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . 

# ***GREEDY BEST FIRST SEARCH***

In [ ]:
import time
from collections import deque
from heapq import heappop, heappush


def greedy_best_first_graph_search(problem, h=None, verbose=False):
    """Greedy best-first search with optional verbose output."""
    h = h or problem.h
    node = Node.root(problem.init_state)
    frontier = [(h(node), id(node), node)]  # Priority queue ordered by h(n)
    explored = set()
    max_frontier_size = 1

    visualizer = Visualizer(problem) if verbose else None

    if verbose:
        visualizer.visualize([node])

    start_time = time.time()  # Start the timer

    while frontier:
        _, _, node = heappop(frontier)
        if problem.goal_test(node.state):
            end_time = time.time()  # End the timer
            elapsed_time = end_time - start_time
            return solution(node), max_frontier_size, elapsed_time

        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored and all(c.state != child.state for _, _, c in frontier):
                heappush(frontier, (h(child), id(child), child))
                if problem.goal_test(child.state):
                    end_time = time.time()  # End the timer
                    elapsed_time = end_time - start_time
                    return solution(child), max_frontier_size, elapsed_time

        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize([n for _, _, n in frontier])

    return None, max_frontier_size, 0  # If no solution, return 0 for elapsed time


# Testing Greedy Best-First Search
obstacles = [(1, 1), (4,2), (3, 3)]
problem = TreasureHunt(grid_size=(20, 20), initial_state=(0, 0), treasure_location=(19, 19),obstacles=obstacles)
solution_path, max_frontier_size, elapsed_time = greedy_best_first_graph_search(problem, h=h_misplaced_tiles, verbose=True)
print(f"Greedy Best-First Search Solution: {actions}")
print(f"Cost: {cost}")
print(f"Elapsed Time: {elapsed_time}")

NameError: name 'h_misplaced_tiles' is not defined

# ***simulated anealing***

In [ ]:
from math import exp
from itertools import count
from random import choice, random
import time

def simulated_annealing(problem, schedule, verbose=False):
    """Simulated annealing search implementation for TreasureHunt."""
    current_state = problem.init_state
    # Define a heuristic function within the simulated_annealing function
    def heuristic(state):
        """Calculates the Manhattan distance to the treasure."""
        return abs(state[0] - problem.treasure_location[0]) + abs(state[1] - problem.treasure_location[1])

    # Use problem.init_state (initial state) instead of the Problem class itself
    current_value = heuristic(problem.init_state)  # Use the defined heuristic function

    if verbose:
        visualizer = Visualizer(problem)  # Assuming Visualizer is defined for TreasureHunt

    start_time = time.time()  # Add start time

    for t in count():
        if verbose:
            # Create a Node object for visualization
            node = Node(current_state, None, None, 0)
            visualizer.visualize([node])  # Pass the Node object to visualize

        T = schedule(t)
        if problem.goal_test(current_state) or T == 0:  # Stop if treasure is found or temperature is 0
            end_time = time.time()  # Add end time
            elapsed_time = end_time - start_time
            print("Elapsed time:", elapsed_time)  # Print elapsed time
            return current_state

        # Get possible actions (movements)
        possible_actions = problem.actions(current_state)
        # Choose a random action
        action = choice(possible_actions)
        # Get the next state after applying the action
        next_state = problem.result(current_state, action)
        # Calculate the heuristic value of the next state
        next_value = heuristic(next_state)  # Use the defined heuristic function
        # Calculate the change in heuristic value
        delta = current_value - next_value

        # Decide whether to accept the next state
        if delta > 0 or random() < exp(delta / T):
            current_state, current_value = next_state, next_value

obstacles = [(1, 1), (4, 2), (3, 3)]
problem = TreasureHunt(grid_size=(20, 20), initial_state=(0, 0), treasure_location=(19, 19),obstacles=obstacles)
schedule = lambda t: max(0.01, min(1, 1 - 0.001 * t))
solution_path, max_frontier_size = bfs_search(problem, verbose=True)

# Run simulated annealing with verbose visualization
final_state = simulated_annealing(problem, schedule, verbose=True)

if solution_path:
    actions, cost = solution_path
    print("Solution found with actions:", actions)
    print("Total cost:", cost)
else:
    print("No solution found.")
print("Final state (solution):", final_state)

# **UCS**

In [ ]:
import time
import heapq
from itertools import count

def ucs_search(problem, verbose=False):
    """Uniform-Cost Search algorithm."""
    node = Node.root(problem.init_state)
    counter = count()
    frontier = [(0, next(counter), node)]
    explored = set()
    max_frontier_size = 1

    visualizer = Visualizer(problem) if verbose else None

    if verbose:
        visualizer.visualize([node])

    start_time = time.time()

    while frontier:
        _, _, node = heapq.heappop(frontier)
        if problem.goal_test(node.state):
            end_time = time.time()
            execution_time = end_time - start_time
            print(f"Algorithm executed in {execution_time:.4f} seconds.")
            return solution(node), max_frontier_size

        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            child_cost = child.path_cost
            if child.state not in explored and all(c[2].state != child.state for c in frontier):
                heapq.heappush(frontier, (child_cost, next(counter), child))
            elif any(c[2].state == child.state and c[0] > child_cost for c in frontier):
                frontier = [(cost, seq, n) if n.state != child.state else (child_cost, next(counter), child)
                            for cost, seq, n in frontier]
                heapq.heapify(frontier)

        max_frontier_size = max(max_frontier_size, len(frontier))

        if verbose:
            visualizer.visualize([n[2] for n in frontier])

    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Algorithm execute in {execution_time:.4f} seconds.")

    return None, max_frontier_size

obstacles = [(1, 1), (4, 2), (3, 3)]
problem = TreasureHunt(grid_size=(20, 20), initial_state=(0, 0), treasure_location=(19, 19),obstacles=obstacles)
solution_path, max_frontier_size = ucs_search(problem, verbose=True)

if solution_path:
    actions, cost = solution_path
    print("Solution found with actions:", actions)
    print("Total cost:", cost)
else:
    print("No solution found.")


# **IDS**

In [ ]:
import time

def ids_search(problem, verbose=False):
    """Iterative Deepening Search algorithm."""
    visualizer = Visualizer(problem) if verbose else None

    def dls(node, depth):
        """Depth-Limited Search helper function."""
        if problem.goal_test(node.state):
            return solution(node), max_frontier_size

        if depth == 0:
            return None, max_frontier_size

        explored.add(node.state)
        for action in problem.actions(node.state):
            child = Node.child(problem, node, action)
            if child.state not in explored:
                if verbose:
                    visualizer.visualize([child])
                result, _ = dls(child, depth - 1)
                if result:
                    return result, max_frontier_size

        return None, max_frontier_size

    start_time = time.time()

    depth = 0
    while True:
        node = Node.root(problem.init_state)
        explored = set()
        max_frontier_size = 1

        if verbose:
            visualizer.visualize([node])

        result, max_frontier_size = dls(node, depth)
        if result:
            end_time = time.time()
            execution_time = end_time - start_time
            print(f"Algorithm execute in {execution_time:.4f} seconds.")
            return result, max_frontier_size

        depth += 1

    # In case of failure to find solution, still print the time
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Algorithm executed in {execution_time:.4f} seconds.")

    return None, max_frontier_size

obstacles = [(1, 1), (4, 2), (3, 3)]
problem = TreasureHunt(grid_size=(20, 20), initial_state=(0, 0), treasure_location=(19, 19),obstacles=obstacles)
solution_path, max_frontier_size = ids_search(problem, verbose=True)

if solution_path:
    actions, cost = solution_path
    print("Solution found with actions:", actions)
    print("Total cost:", cost)
else:
    print("No solution found.")


## *Genetic mutation*

In [ ]:
import random

In [ ]:
class Node:
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost

    def __lt__(self, other):
        """Define how to compare nodes using their path cost."""
        return self.path_cost < other.path_cost

    def root(cls, state):
        return cls(state)

    @classmethod
    def child(cls, problem, parent, action):
        return cls(action, parent, parent.path_cost + 1)

def solution(node):
    actions = []
    cost = node.path_cost
    while node.parent is not None:
        actions.append(node.action)
        node = node.parent
    actions.reverse()
    return actions, cost

In [ ]:
class TreasureHunt:
    def __init__(self, grid_size=(32, 32), treasure_location=(31, 31), initial_state=(0, 0), obstacles=None):
        self.grid_size = grid_size
        self.treasure_location = treasure_location
        self.init_state = initial_state
        self.obstacles = obstacles if obstacles is not None else []

    def actions(self, state):
        possible_actions = ['up', 'down', 'left', 'right']
        valid_actions = []

        for action in possible_actions:
            new_row, new_col = self.result(state, action)
            if (0 <= new_row < self.grid_size[0] and
                0 <= new_col < self.grid_size[1] and
                (new_row, new_col) not in self.obstacles):
                valid_actions.append(action)

        return valid_actions

    def result(self, state, action):
        row, col = state

        if action == 'up':
            row -= 1
        elif action == 'down':
            row += 1
        elif action == 'left':
            col -= 1
        elif action == 'right':
            col += 1

        return row, col

    def goal_test(self, state):
        return state == self.treasure_location

    def step_cost(self, state, action):
        return 1  # Assume each move has a cost of 1

In [ ]:
def _treasure_hunt_visualizer(problem, state):
    grid = [['.' for _ in range(problem.grid_size[1])] for _ in range(problem.grid_size[0])]
    grid[problem.treasure_location[0]][problem.treasure_location[1]] = 'T'
    grid[state[0]][state[1]] = 'A'

    for obstacle in problem.obstacles:
        grid[obstacle[0]][obstacle[1]] = 'X'

    for row in grid:
        print(' '.join(row))

_visualizers[TreasureHunt] = _treasure_hunt_visualizer

In [ ]:
def genetic_algorithm(population, fitness_fn, gene_pool, f_thres, ngen=1000, pmut=0.1, visualizer=None):
    """Genetic algorithm that visualizes each generation."""
    for i in range(ngen):

        population = [
            mutate(recombine(*select(2, population, fitness_fn)), gene_pool, pmut)
            for _ in range(len(population))
        ]

        fittest_individual = max(population, key=fitness_fn)
        if fitness_fn(fittest_individual) == f_thres:
            if visualizer:
                print(f"Solution found at generation {i + 1}")
                visualizer.visualize_state(fittest_individual)
            return fittest_individual

        if visualizer:
            print(f"Generation {i + 1}")
            visualizer.visualize_state(fittest_individual)

    return max(population, key=fitness_fn)

In [ ]:
def recombine(x, y):
    x1, y1 = x.state
    x2, y2 = y.state
    if x1 == x2:
        new_state = (x1, y2)
    else:
        new_state = (x2, y1)
    return Node(state=new_state, parent=None, action=None, path_cost=0)

In [ ]:
def mutate(node, gene_pool, pmut):
    """Mutate a node's state with probability pmut"""
    if random.uniform(0, 1) < pmut:
        new_state = random.choice(gene_pool)
        return Node(state=new_state, parent=None, action=None, path_cost=0)
    return node

In [ ]:
def init_population(pop_number, gene_pool, state_length):
    """Initializes population for genetic algorithm"""
    population = []
    for _ in range(pop_number):
        state = random.choice(gene_pool)
        population.append(Node(state=state, parent=None, action=None, path_cost=0))
    return population

In [ ]:
def memoize(fn, attr_name):
    """Memoizes function results by storing them in an attribute of the problem object."""
    cache = {}
    def memoized_fn(node, problem):
        if not hasattr(problem, attr_name):
            setattr(problem, attr_name, {})
        cache = getattr(problem, attr_name)
        if node.state not in cache:
            cache[node.state] = fn(node, problem)
        return cache[node.state]

    return memoized_fn

In [ ]:
def calculate_fitness(node):
    """Calculate fitness as negative Manhattan distance to the treasure."""
    agent_x, agent_y = node.state
    treasure_x, treasure_y = treasure_hunt.treasure_location
    return -(abs(agent_x - treasure_x) + abs(agent_y - treasure_y))

In [ ]:
def select(r, population, fitness_fn):
    """Select r individuals from population and return them as a list."""
    tournament = random.sample(population, r)
    return sorted(tournament, key=fitness_fn, reverse=True)[:r]

In [ ]:
treasure_hunt = TreasureHunt(
   grid_size=(20, 20), initial_state=(0, 0), treasure_location=(19, 19),obstacles=obstacles
)

print("Initial state:")
_treasure_hunt_visualizer(treasure_hunt, treasure_hunt.init_state)

gene_pool = [(x, y) for x in range(6) for y in range(6)]
population = init_population(pop_number=10, gene_pool=gene_pool, state_length=2)
f_thres = 0

result = genetic_algorithm(
    population=population,
    fitness_fn=calculate_fitness,
    gene_pool=gene_pool,
    f_thres=f_thres,
    ngen=50,
    pmut=0.1
)

print("\nFinal state:")
_treasure_hunt_visualizer(treasure_hunt, result.state)

In [ ]:
import time

def measure_time(func, *args, **kwargs):
    """
    Measures and prints the execution time of a function.

    Parameters:
        func: The function to measure.
        *args: Positional arguments for the function.
        **kwargs: Keyword arguments for the function.

    Returns:
        result: The result of the function call.
    """
    start_time = time.time()
    result = func(*args, **kwargs)
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time for {func.__name__}: {execution_time:.4f} seconds")

    return result

f_thres = 0

result = measure_time(
    genetic_algorithm,
    population,
    calculate_fitness,
    gene_pool,
    f_thres,
    ngen=100,
    pmut=0.1
)